In [1]:
# (1) 데이터로드
import pandas as pd
import numpy as np
from keras.utils import np_utils

import warnings
warnings.filterwarnings(action='ignore')

# 파일읽기

# gcc6_0~3
gcc7_1_32 = pd.read_csv("binutils_gcc3~9_op0~4_csv/gcc7_1_32.csv", index_col=0)

# 형태 출력
print(gcc7_1_32.shape)
gcc7_1_32.head()

Using TensorFlow backend.


(1875089, 2)


,hex,bin
1,41,1
2,57,0
3,41,0
4,56,0
5,41,0


In [2]:
# (2-1) 데이터체크 - hex(16진수)가 256 label을 가져야 dummies 변환 가능 

# 16진수 256개 종류가 있어서 pd.get_dummies 사용 가능.
print(len(gcc7_1_32['hex'].unique()))

256


In [3]:
# (3) get_dummies 변환 

# 훈련데이터 (gcc 최적화버전 0, 1, 2 one hot encoding)
gcc7_1_32_onehot = pd.get_dummies(gcc7_1_32)

print('원핫인코딩완료')

원핫인코딩완료


In [5]:
# (4) 데이터 체크 - 1, 0 비율 ==> 1이 함수의 갯수를 뜻함

# 정답 데이터 1, 0 비율 확인  ==> 1이 함수의 갯수를 뜻함
v_count = gcc7_1_32_onehot['bin'].value_counts()

print(v_count, '\n함수의 평균 길이는', v_count[0]/v_count[1])


0    1871751
1       3338
Name: bin, dtype: int64 
함수의 평균 길이는 560.7402636309167


In [7]:
# (7) 훈련데이터, 훈련라벨 분리 

# 훈련 데이터, 훈련 라벨
x_train_gcc7_1_32 = gcc7_1_32_onehot.iloc[:,1:].to_numpy()
y_train_gcc7_1_32 = gcc7_1_32_onehot['bin'].to_numpy()

print(x_train_gcc7_1_32.shape, y_train_gcc7_1_32.shape)

(1875089, 256) (1875089,)


In [8]:

# (8) 훈련데이터, 훈련라벨 reshape 작업 => # ex - (605, 1, 256)

x_train_gcc7_1_32 = x_train_gcc7_1_32.reshape(x_train_gcc7_1_32.shape[0], 1, x_train_gcc7_1_32.shape[1])
y_train_gcc7_1_32 = y_train_gcc7_1_32.reshape(y_train_gcc7_1_32.shape[0], 1, 1)

## test data
print(x_train_gcc7_1_32.shape, y_train_gcc7_1_32.shape)

(1875089, 1, 256) (1875089, 1, 1)


In [11]:
# (9) 랜덤으로 X chunk씩 뽑기.

from random import *

X=10000
train_range = []
_,s,d = x_train_gcc7_1_32.shape # 행, 1, 256

# numpy 배열 초기화
rs_x = np.zeros(shape = (10, X, s, d))
rs_y = np.zeros(shape = (10, X, s, 1))

for i in range(0,10):
    n = randint(0, len(x_train_gcc7_1_32)-X)
    rs_x[i] = (x_train_gcc7_1_32[n:n+X])
    rs_y[i] = (y_train_gcc7_1_32[n:n+X])
#    print(rs_x[i], '\n')
    train_range.append((n,n+X))

data_10000x = np.hstack(rs_x).reshape(X*10, s, d)
data_10000y = np.hstack(rs_y).reshape(X*10, s, 1)

print(data_10000x.shape, data_10000y.shape, '\n')
print('학습범위', train_range)

(100000, 1, 256) (100000, 1, 1) 

학습범위 [(1604384, 1614384), (254252, 264252), (648685, 658685), (1853687, 1863687), (1105279, 1115279), (395285, 405285), (592668, 602668), (76201, 86201), (1839701, 1849701), (861194, 871194)]


In [15]:
# (10) 양방향 LSTM 모델링 작업
from keras.models import Model, Sequential
from keras.layers import SimpleRNN, Input, Dense, LSTM
from keras.layers import Bidirectional, TimeDistributed

# 교차검증 kfold
from sklearn.model_selection import KFold

# 학습
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(patience = 3) # 조기종료 콜백함수 정의

# Accuracy, Precision, Recall, F1-Score
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

# K-Fold 객체 생성
kf = KFold(n_splits=10, shuffle=False, random_state=None) # KFold non shuffle 버전

# Confusion Matrix
from sklearn.metrics import confusion_matrix

# ROC Curve
from sklearn.metrics import roc_auc_score

# 최종 평가지표들 평균용
accuracy, recall, precision, f1score, cm = [], [], [], [], []

# 11. 교차검증 kfold - k.split - 10회
for train, validation in kf.split(data_10000x, data_10000y):
    xInput = Input(batch_shape=(None, 1, 256)) # 
    xBiLstm = Bidirectional(LSTM(16, return_sequences=True), merge_mode = 'concat')(xInput)
    xOutput = TimeDistributed(Dense(1, activation ='sigmoid'))(xBiLstm) # 각 스텝에서 cost가 전송되고, 오류가 다음 step으로 전송됨.

    model1 = Model(xInput, xOutput)
    model1.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    #print(model1.summary())
    print('======Training stage======')
    model1.fit(data_10000x[train],
               data_10000y[train],
               epochs = 3,
               batch_size = 32,
               callbacks=[early_stopping])
    #k_accuracy = '%.4f' %(model1.evaluate(data_10000x[validation], data_10000y[validation])[1])

# 12. 교차검증결과 predict    
    # predict 값
    k_pr = model1.predict(data_10000x[validation])
    
    # 테스트 predict 결과들 비교 (평가지표 보기위함)
    pred = np.round(np.array(k_pr).flatten().tolist())
    y_test = np.array(data_10000y[validation]).flatten().tolist()
    
# 13. 평가지표들 출력
    ## 평가지표들
    k_accuracy = float(accuracy_score(y_test, pred))
    k_recall =  float(recall_score(y_test, pred))
    k_precision = float(precision_score(y_test, pred))
    k_f1_score = float(f1_score(y_test, pred))
    #k_cm = float(confusion_matrix(y_test, pred))
    
    print('accuracy_score', k_accuracy)
    print('recall_score', k_recall)
    print('precision_score', k_precision)
    print('f1_score', k_f1_score)
    #print('\nconfusion_matrix\n', k_cm)

    accuracy.append(k_accuracy)
    recall.append(k_recall)
    precision.append(k_precision)
    f1score.append(k_f1_score)
    #cm.append(k_cm)
#    print('roc_curve 면적', roc_auc_score(y_test, pred))

# 14. 최종 결과지표
print('\nK-fold cross validation Accuracy: {}'.format(accuracy))
print('\nK-fold cross validation Recall: {}'.format(recall))
print('\nK-fold cross validation Precision: {}'.format(precision))
print('\nK-fold cross validation F1-Score: {}'.format(f1score))
#print('\nK-fold cross validation ConfusionMatrix: {}'.format(cm))

======Training stage======
Epoch 1/3
90000/90000 [==============================] - 8s 84us/step - loss: 0.0509 - accuracy: 0.9971
Epoch 2/3
90000/90000 [==============================] - 8s 85us/step - loss: 0.0166 - accuracy: 0.9975
Epoch 3/3
90000/90000 [==============================] - 10s 106us/step - loss: 0.0164 - accuracy: 0.9975
accuracy_score 0.9977
recall_score 0.0
precision_score 0.0
f1_score 0.0
======Training stage======
Epoch 1/3
90000/90000 [==============================] - 8s 91us/step - loss: 0.0513 - accuracy: 0.9972
Epoch 2/3
90000/90000 [==============================] - 8s 85us/step - loss: 0.0160 - accuracy: 0.9975
Epoch 3/3
90000/90000 [==============================] - 8s 85us/step - loss: 0.0159 - accuracy: 0.9975
accuracy_score 0.9976
recall_score 0.0
precision_score 0.0
f1_score 0.0
======Training stage======
Epoch 1/3
90000/90000 [==============================] - 8s 92us/step - loss: 0.0511 - accuracy: 0.9968
Epoch 2/3
90000/90000 [======================